# Simple Stop Watch


## Particle CLI

### Installation

In [1]:
%%capture
!bash <( curl -sL https://particle.io/install-cli )

# path to the particle cli. May be environment dependent.
particle_cli = "/root/bin/particle"

### Utility functions

In [2]:
import re
import subprocess

# regular expression to strip ansi control characters
ansi = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# decode byte string and strip ansi control characters
def decode_bytes(byte_string):
    if isinstance(byte_string, bytes):
        result = byte_string.decode("utf-8")
    return ansi.sub("", result)

# streamline call to the particle-cli
def particle(args):
    process = subprocess.run(["/root/bin/particle"] + args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
    process.stdout = decode_bytes(process.stdout)
    process.stderr = decode_bytes(process.stderr)
    return process

# print the default help message
print(particle(["help"]).stderr)

Welcome to the Particle Command Line Interface!
Version 2.8.1
https://github.com/particle-iot/particle-cli

Usage: particle <command>
Help:  particle help <command>

Commands:
  binary      Inspect binaries
  call        Call a particular function on a device
  cloud       Access Particle cloud functionality
  compile     Compile a source file, or directory using the cloud compiler
  config      Configure and switch between multiple accounts
  device      Manipulate a device
  doctor      Put your device back into a healthy state
  flash       Send firmware to your device
  function    Call functions on your device
  get         Retrieve a value from your device
  identify    Ask for and display device ID via serial
  keys        Manage your device's key pair and server public key
  library     Manage firmware libraries
  list        Display a list of your devices, as well as their variables and functions
  login       Login to the cloud and store an access token locally
  logout      

### Login to Particle

In [3]:
import getpass

# prompt for username and password
username = getpass.getpass(prompt="Username: ")
password = getpass.getpass(prompt="Password: ")

# attempt login
output = particle(["login", "--username", username, "--password", password])

# report results
if output.returncode:
    print(f"Return code = {output.returncode}")
    print(output.stderr)
else:
    print(output.stdout)

Username: ··········
Password: ··········
> Successfully completed login!



### Select a device

The following cell downloads a list of all user devices and creates a list of device names. Here we choose the first name in the list for the rest of this notebook. If this is not the device to be used, then modify this cell accordingly.

In [4]:
devices = [line.split()[0] for line in particle(["list"]).stdout.splitlines()]
device_name = devices[0]
print(particle(["list", device_name]).stdout)

jck_argon_01 [e00fce68eaceb1faa7cf7193] (Argon) is online



## Project: Simple Stop Watch


### Create Project

In [5]:
print(particle(["project", "create", "--name", "myproject", "."]).stdout)

Initializing project in directory myproject...
> A new project has been initialized in directory myproject



### Change working directory

The Particle CLI assumes one is working in the top  project directory.

In [6]:
%cd myproject

/content/myproject


### Add relevant libraries

In [7]:
print(particle(["library", "add", "Grove_4Digit_Display"]).stdout)
print(particle(["library", "add", "clickButton"]).stdout)

> Library Grove_4Digit_Display 1.0.2 has been added to the project.
> To get started using this library, run particle library view Grove_4Digit_Display to view the library documentation and sources.

> Library clickButton 1.0.9 has been added to the project.
> To get started using this library, run particle library view clickButton to view the library documentation and sources.



### Create source file

In [12]:
%%writefile src/myproject.ino

#define CLK D2     /* display clock pin */
#define DIO D3     /* display data pin */
#define DIGITS 4   /* number of display digits */

#include "Grove_4Digit_Display.h"
#include "clickButton.h"

/* display */
TM1637 tm1637(CLK, DIO);

/* button */
const int btn = 4;
ClickButton button(btn, HIGH);

/* stopwatch state */
unsigned long curr_time;
unsigned long prev_time;
unsigned long display_time;
boolean running;

void setup() {
    /* setup display */
    tm1637.init();
    tm1637.set(BRIGHT_TYPICAL);
    tm1637.point(POINT_ON);

    /* setup button */
    pinMode(D4, INPUT);
    button.debounceTime = 0;
    button.multiclickTime = 250;
    button.longClickTime = 1000;

    /* setup stopwatch */
    prev_time = millis();
    display_time = 0;
    running = FALSE;

    /* record event */
    Particle.publish("StopWatch", "Starting", PUBLIC);
}

void loop() {
    button.Update();
    if (button.clicks > 0) {
        Particle.publish("StopWatch", "short button press", PUBLIC);
        running = !running;
    } else if (button.clicks < 0) {
        Particle.publish("StopWatch", "long button press", PUBLIC);
        display_time = 0;
    }
    if (running) {
        curr_time = millis();
        display_time += curr_time - prev_time;
    } else {
        curr_time = millis();
    }
    prev_time = curr_time;
    display(display_time / 10); /* displaying 100th's of seconds */
}

void display(unsigned int number) {
    for (int i = 0; i < 4; i++) {
        int digit = DIGITS - 1 - i;
        tm1637.display(digit, number % 10);
        number /= 10;
    }
}

Overwriting src/myproject.ino


### Compiling

In [13]:
print(particle(["compile", "argon", "--saveTo", "myproject.bin"]).stdout)


Compiling code for argon

Including:
    src/myproject.ino
    project.properties

attempting to compile firmware
downloading binary from: /v1/binaries/5f90eb90e4e856c719230896
saving to: myproject.bin
Memory use:
   text	   data	    bss	    dec	    hex	filename
   8492	    108	   1116	   9716	   25f4	/workspace/target/workspace.elf

Compile succeeded.
Saved firmware to: /content/myproject/myproject.bin



### Flash firmware

In [14]:
print(particle(["flash", device_name, "myproject.bin"]).stdout)

Including:
    myproject.bin

attempting to flash firmware to your device jck_argon_01
Flash device OK: Update started

Flash success!

